# This file is used to train and test the model.

In [17]:
import torch
import numpy as np
from tqdm import tqdm
import pandas as pd
from sc_model import SC_LSTM as Model

# Device init

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    !nvidia-smi
    print(torch.cuda.get_device_name(0))

else:
    print("No GPU :(")

Sun Jul  2 02:27:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti      Off| 00000000:01:00.0  On |                  N/A |
| N/A   48C    P0               16W /  N/A|    839MiB /  4096MiB |     21%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [19]:
device = 'cpu'

## Loading the data

In [20]:
from Data.Preprocessor import Preprocessor

preproc = Preprocessor()

data = pd.read_csv('../Data/schedule_v3.csv')
data

,Label Number,Duration,Importance,Start Time,Date
0,3,75,1,6:30,22/06/2023
1,5,90,2,8:15,22/06/2023
2,0,30,0,10:00,22/06/2023
3,1,30,1,10:45,22/06/2023
4,6,15,0,11:30,22/06/2023
...,...,...,...,...,...
967,4,90,2,18:15,01/09/2023
968,1,30,1,19:30,01/09/2023
969,6,45,0,20:45,01/09/2023
970,3,45,1,22:00,01/09/2023


In [21]:
input_data, type_vector, output_vector = preproc.preprocess(data)

In [22]:
from sklearn.preprocessing import MinMaxScaler

# Normalize input data
scaler = MinMaxScaler()
input_data = scaler.fit_transform(input_data)

## Creating the dataset

In [23]:
from torch.utils.data import Dataset, DataLoader

class ScheduleDataset(Dataset):
    def __init__(self, input_data, type_vector, output_vector, transform=None):
        self.input_data = input_data
        self.type_vector = type_vector
        self.output_vector = output_vector
        self.transform = transform

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        return self.input_data[idx], self.type_vector[idx], self.output_vector[idx]

In [24]:
dataset = ScheduleDataset(input_data, type_vector, output_vector)

## Create dataloaders

In [25]:
batch_size = 1
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
test_size = dataset_size - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# also conver
train_dataloader = torch.utils.data.DataLoader2(train_dataset, batch_size=batch_size)
valid_dataloader = torch.utils.data.DataLoader2(test_dataset, batch_size=batch_size)

## Create free time slots generator

In [26]:
from Data.GeneratorOfAvailableTimeslots import GeneratorOfAvailableTimeslots

time_slots_gen = GeneratorOfAvailableTimeslots(5)
time_slots = time_slots_gen.generate_available_timeslots()

time_slots      # TODO: why all the intervals are closed?

# TODO THIS IS ONLY FOR TESTING
single_interval = time_slots[0]
single_interval

(13, 2)
(13, 2)
(13, 2)
(13, 2)
(13, 2)
(13, 2)
(13, 2)
(13, 2)
(13, 2)
(13, 2)
(13, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(10, 2)
(10, 2)
(10, 2)
(10, 2)
(10, 2)
(10, 2)
(10, 2)
(10, 2)
(10, 2)
(10, 2)
(5, 2)
(5, 2)
(5, 2)
(5, 2)
(5, 2)
(6, 2)
(6, 2)
(6, 2)
(6, 2)
(6, 2)
(6, 2)


array([[0.        , 0.69680259],
       [0.69680259, 0.7555514 ],
       [0.7555514 , 0.99597873],
       [0.99597873, 0.9972507 ],
       [0.9972507 , 0.99912933],
       [0.99912933, 0.99965211],
       [0.99965211, 0.99971676],
       [0.99971676, 0.99971984],
       [0.99971984, 0.99993107],
       [0.99993107, 0.99999218],
       [0.99999218, 0.9999962 ],
       [0.9999962 , 1.        ]])

## Init the model

In [27]:
# check if the dataloader works and get the input and output sizes
in_features = 0
out_features = 0
for i, (features, _, ans) in enumerate(train_dataloader):
    in_features = features.shape[1]
    out_features = 3             # TODO: change to ans.shape[1]
    break

n_layers = 1
hidden_size = 30

# Create the model
SC_LSTM = Model(in_features, n_layers, hidden_size, out_features, batch_size).to(device)

# Testing the model

In [28]:
# Configure hyper-parameters
epochs = 1
learning_rate = 0.001
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(SC_LSTM.parameters(), lr=learning_rate)

history = []
loss_accomulator = []

In [29]:
# Firstly train the LSTM using only reschedulable tasks
SC_LSTM.train()
SC_LSTM.train_lstm()

for epoch in range(0, epochs):
    for i, (X, task_type, Y) in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", leave=False, colour='green')):

        # Convert X to the correct type
        X = torch.Tensor(X).type(torch.float32)

        # Forward only for reschedulable tasks
        if task_type[0] == "resched":

            Y_pred = SC_LSTM.forward(X, task_type=task_type, free_time_slots=single_interval).to(device)


        # # Calculate loss
        # loss = loss_func(Y_pred.view(-1, ), Y.view(-1, ))
        #
        # # Backward pass
        # loss.backward(retain_graph=True)
        # optimizer.step()
        # optimizer.zero_grad()
        #
        # # Append loss to list
        # loss_accomulator.append(loss.item())
        #
        # history.append(np.mean(loss_accomulator))
        # loss_accomulator = []

In [30]:
# Configure hyper-parameters
epochs = 1
learning_rate = 0.001
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(SC_LSTM.parameters(), lr=learning_rate)

history = []
loss_accomulator = []

In [32]:
# Now train the injector using non-reschedulable tasks
SC_LSTM.train()
SC_LSTM.train_injector()

for epoch in range(0, epochs):
    for i, (X, task_type, Y) in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", leave=False, colour='green')):

        # Convert X to the correct type
        X = torch.Tensor(X).type(torch.float32)

        # Forward only for non-reschedulable tasks
        if task_type[0] == "non-resched":

            Y_pred = SC_LSTM.forward(X, task_type=task_type, free_time_slots=single_interval).to(device)


            #TODO how to convert X to Y ~ (start, end, refr)
            # # Calculate loss
            # loss = loss_func(Y_pred.view(-1, ), Y.view(-1, ))
            #
            # # Backward pass
            # loss.backward(retain_graph=True)
            # optimizer.step()
            # optimizer.zero_grad()
            #
            # # Append loss to list
            # loss_accomulator.append(loss.item())
            #
            # history.append(np.mean(loss_accomulator))
            # loss_accomulator = []